In [1]:
import zipfile

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2023-11-07 02:53:30--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c0d::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.2’

10_food_classes_10_ 100%[===================>] 160.74M   267MB/s    in 0.6s    

2023-11-07 02:53:30 (267 MB/s) - ‘10_food_classes_10_percent.zip.2’ saved [168546183/168546183]



In [3]:
# unzip the dataset

zip_ref = zipfile.ZipFile("/content/10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [4]:
import os

for dirpath,dirnames,filenames in os.walk("/content/10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '/content/10_food_classes_10_percent'.
There are 10 directories and 0 images in '/content/10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '/content/10_food_classes_10_per

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (224,224)
batch_size = 32

train_dir = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"

In [6]:
train_datagen = ImageDataGenerator(rescale = 1/225.)
test_datagen = ImageDataGenerator(rescale = 1/225.)

In [7]:
train_data = train_datagen.flow_from_directory(
                            train_dir,
                            target_size = image_size,
                            batch_size = batch_size,
                            class_mode= "categorical"
)

test_data = test_datagen.flow_from_directory(
                            test_dir,
                            target_size = image_size,
                            batch_size = batch_size,
                            class_mode= "categorical"
)

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [8]:
import datetime

def create_tensorflow_callback(dir_name,experiment_name):

  log_dir = dir_name + "/" +experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  tensorflow_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

  return tensorflow_callback

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [10]:
resnet50_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5"

resnet101_url = "https://tfhub.dev/google/imagenet/resnet_v1_101/classification/5"

resnet152_url = "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/5"

In [11]:
def create_model(model_url,num_classes = 10):

  feature_extractor_layer = hub.KerasLayer(
                                      model_url,
                                      trainable = False,
                                      name = "feature_extraction_layer",
                                      input_shape = image_size +(3,)
    )

  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes,activation = "softmax",name = "output_layer")
    ])

  return model

In [12]:
model = create_model(resnet50_url,num_classes = 10)

In [13]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
history = model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data),
    callbacks = [create_tensorflow_callback(dir_name = "tensorflo_hub",experiment_name = "resnet_50")]
)

Epoch 1/5
 5/24 [=====>........................] - ETA: 1:10 - loss: 3.5174 - accuracy: 0.1500